# 1. Appending new rows to DataFrames

## 1.1. Append without using `append()` (using `loc`)

In [1]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x : '%.3f' % x)
pd.set_option('max_columns', None)

In [8]:
df = pd.DataFrame(columns=['a','b'])
df.head()

,a,b


### 1.1.1. Add data as 'list'

In [9]:
df.loc[0] = [1,2]
df.head()

,a,b
0,1,2


### 1.1.2 Add data as 'dict'

In [10]:
df.loc[len(df)] = {'b' : 'ㅎ', 'a': 'ㅋ'}
df.head()

,a,b
0,1,2
1,ㅋ,ㅎ


### 1.1.3. Add data as 'Series'

In [11]:
df.loc["yay"] = pd.Series({'a':'ㅋ','b':'ㅎ'})
df.tail()

,a,b
0,1,2
1,ㅋ,ㅎ
yay,ㅋ,ㅎ


In [12]:
# 이미 존재하는 index에 넣기
df.loc["yay"] = pd.Series({'a':'1111','b':'2222'})
df.tail()

,a,b
0,1,2
1,ㅋ,ㅎ
yay,1111,2222
